## Практическое задание к уроку № 3 по теме "TensorFlow".

Задание 1.

*Попробуйте улучшить работу нейронной сети (разобранную на уроке), обучавшейся на датасете Fashion-MNIST.  
Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?*

Сделаем необходимые импорты:

In [1]:
import tensorflow as tf

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

На уроке мы получали метрику accuracy на тестовом датасете в районе 0,87.  
В данном практическом задании попробуем улучшить этот показатель.

In [3]:
train_images = (train_images / 255.0) - 0.5

test_images = (test_images / 255.0) - 0.5

In [4]:
tf.random.set_seed(29)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.BatchNormalization(momentum=0.995),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid')
])

In [7]:
model.compile(optimizer=tf.keras.optimizers.Nadam(beta_2=0.9999),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [8]:
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4509 - accuracy: 0.8362
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3445 - accuracy: 0.8730
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3108 - accuracy: 0.8849
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2859 - accuracy: 0.8926
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2700 - accuracy: 0.8978


In [9]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 723us/step - loss: 0.3374 - accuracy: 0.8821


[0.3373505473136902, 0.882099986076355]

<ins>Вывод:</ins>  
Получили более 0,88. Улучшить метрику позволили следующие действия:  
- Вычитаем 0,5 при первоначальной нормализации, как делали на  
уроке №2. Это позволяет избежать зануления признаков, и они не  
выпадают из обучения;  
- Перенесли нормализацию внутри модели ближе к входному слою,  
подобрали её параметры;  
- Поменяли функцию активации выходного слоя с softmax на сигмоиду;  
- Поменяли метод оптимизации с Adam на Nadam и подобрали его параметры;  
- Сократили количество эпох с 6 до 5.

Задание 2*.

*Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews.  
Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?*

Сделаем необходимые импорты. Ограничим количество уникальных  
слов в обзорах до 20000 самых популярных:

In [10]:
imdb_reviews = tf.keras.datasets.imdb

In [11]:
(X_train, y_train), (X_test, y_test) = imdb_reviews.load_data(num_words=20000)

In [12]:
X_train.shape, y_train.shape

((25000,), (25000,))

In [13]:
X_test.shape, y_test.shape

((25000,), (25000,))

In [14]:
from collections import Counter
import numpy as np
import pandas as pd

Посмотрим, как выглядят данные:

In [15]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [16]:
y_train[:5]

array([1, 0, 0, 1, 0])

Данные состоят из списков с индексами слов,  
составляющих обзор на фильм. Целевой датасет  
содержит информацию о том, хороший фильм или нет,  
то есть имеем задачу бинарной классификации.

Нужно подготовить данные для нейросети, для этого  
представим данные в виде датафрейма, столбцами которого  
будут индексы слов. Данными в ячейках будут числа, представляющие  
количество слов определённого индекса в определённом обзоре.

Создадим множество всех индексов слов датасета:

In [17]:
words = set()
for word_list in X_train:
    words.update(word_list)

Создадим обучающий и тестовый датафреймы:

In [18]:
data = np.zeros((len(X_train), len(words)))
train_df = pd.DataFrame(data=data, columns=list(words))
test_df = train_df.copy()

for i in range(X_train.shape[0]):
    train_df.iloc[i].update(Counter(X_train[i]))
    test_df.iloc[i].update(Counter(X_test[i]))

Посмотрим на получившиеся датасеты:

In [19]:
train_df.head()

,1,2,4,5,6,7,8,9,10,11,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
0,1.0,3.0,15.0,9.0,3.0,2.0,3.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,4.0,15.0,10.0,3.0,2.0,3.0,10.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,2.0,9.0,0.0,4.0,4.0,4.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,23.0,36.0,12.0,8.0,21.0,12.0,12.0,16.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,2.0,6.0,2.0,4.0,2.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
test_df.head()

,1,2,4,5,6,7,8,9,10,11,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
0,1.0,0.0,3.0,2.0,2.0,1.0,2.0,1.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2.0,18.0,9.0,6.0,7.0,6.0,7.0,6.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,33.0,36.0,11.0,23.0,18.0,14.0,8.0,8.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,13.0,3.0,2.0,3.0,2.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,3.0,1.0,3.0,1.0,0.0,3.0,2.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Так как эти датасеты содержат большое количество нулей, то  
конвертируем их в разреженные матрицы для экономии памяти:  

In [21]:
train_df = tf.sparse.from_dense(train_df)
test_df = tf.sparse.from_dense(test_df)

2022-07-08 17:38:09.348083: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3999600000 exceeds 10% of free system memory.
2022-07-08 17:38:10.579132: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3999600000 exceeds 10% of free system memory.
2022-07-08 17:38:13.255794: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3999600000 exceeds 10% of free system memory.
2022-07-08 17:38:14.508940: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3999600000 exceeds 10% of free system memory.


Инициализируем и обучим нейронную сеть:

In [22]:
tf.random.set_seed(29)

imdb_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(train_df.shape[1]), sparse=True),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])  

In [23]:
imdb_model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

In [24]:
imdb_model.fit(train_df, y_train, epochs=2)

Epoch 1/2


/home/shkin/.local/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_2/dense_6/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_2/dense_6/embedding_lookup_sparse/Reshape:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_2/dense_6/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


782/782 [==============================] - 2s 2ms/step - loss: 0.3744 - accuracy: 0.8370
Epoch 2/2
782/782 [==============================] - 2s 2ms/step - loss: 0.2156 - accuracy: 0.9168


In [25]:
imdb_model.evaluate(test_df, y_test)

782/782 [==============================] - 1s 951us/step - loss: 0.3260 - accuracy: 0.8809


[0.32597240805625916, 0.8808799982070923]

<ins>Вывод:</ins>  
Подготовили данные и обучили нейронную сеть на задаче  
бинарной классификации. Получили приемлемую точность 88%.  